<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11n_Shikibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

In [ ]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

In [7]:
model_path = "/content/drive/MyDrive/yolo11n.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [ ]:
import yaml

# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

In [ ]:
import numpy as np
import os
from ultralytics import YOLO

def train_yolo(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    print(f"Training YOLO11 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolo11_training",

        # これまで通り
        iou=iou_threshold,
        mosaic=mosaic,
        mixup=mixup,
        degrees=4,

        # ↓↓↓ ここから“下で定義済みのパラメータ”をそのまま渡す ↓↓↓
        lr0=lr0,
        weight_decay=weight_decay,
        label_smoothing=label_smoothing,
        copy_paste=copy_paste,
        translate=translate,
        close_mosaic=close_mosaic,
        perspective=perspective,
        shear=shear,
        fliplr=fliplr,
        flipud=flipud,
        rect=rect,
        cos_lr=cos_lr,
        multi_scale=multi_scale,
        patience=patience,
        seed=seed,
        fl_gamma=1.5,
        optimizer="AdamW"
    )
    print(f"Training complete. Results saved to {output_dir}")


# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLO の学習実行
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)

In [ ]:
# === Stage 2: 高解像度(1280)で短期微調整 ===
# 使い方：Stage-1 学習(960px)が完了したあと、このセルを実行

import os, time

def find_latest_best(project_dir, run_name_contains="yolo11_training"):
    """
    project_dir(例: 'runs_experiment') 以下から
    yolo11_training*/weights/best.pt のうち最も新しいものを返す
    """
    latest = None
    latest_mtime = -1
    for root, dirs, files in os.walk(project_dir):
        if root.endswith("weights") and "best.pt" in files and run_name_contains in root:
            p = os.path.join(root, "best.pt")
            m = os.path.getmtime(p)
            if m > latest_mtime:
                latest = p
                latest_mtime = m
    if latest is None:
        # フォールバック（最初のラン名想定）
        cand = os.path.join(project_dir, "yolo11_training", "weights", "best.pt")
        if os.path.exists(cand):
            return cand
        raise FileNotFoundError(f"No best.pt found under {project_dir}")
    return latest

# 直近の Stage-1 の best.pt を起点にする
stage2_model_path = find_latest_best(output_dir)
print("Stage2 base model:", stage2_model_path)

# 高解像度で短期微調整
img_size_stage2   = 1280
epochs_stage2     = 50

# VRAMに合わせて自動で少しバッチを縮小（960→1280 は面積比で ~0.56）
batch_size_stage2 = max(8, int(batch_size * (img_size / img_size_stage2) ** 2))
print("batch_size_stage2 =", batch_size_stage2)

# 一時的に lr0 と multi_scale を上書きしてコール（関数はそのまま再利用）
_lr0_bak = lr0
_multi_bak = multi_scale
try:
    lr0 = 0.001        # 微調整向けに学習率を下げる
    multi_scale = False  # 1280固定で回す

    train_yolo(
        model_path=stage2_model_path,   # Stage-1 の best.pt から継続
        data_yaml=data_yaml,
        epochs=epochs_stage2,           # 50
        batch_size=batch_size_stage2,   # 自動調整済み
        img_size=img_size_stage2,       # 1280
        output_dir=output_dir,          # 例: "runs_experiment"
        iou_threshold=iou_threshold,    # 0.5
        mosaic=0.0,                     # rect=True なので実質OFF
        mixup=0.0
    )
finally:
    # 以降のセルに影響しないように元に戻す
    lr0 = _lr0_bak
    multi_scale = _multi_bak


In [10]:
# best.pt のパス
best_model_path = 'runs_experiment/yolo11_training/weights/best.pt'

# best.pt が存在すれば自動でダウンロード
if os.path.exists(best_model_path):
    files.download(best_model_path)
else:
    print("best.pt が見つかりませんでした。")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>